In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
best_model = '/kaggle/input/pss5e1-h2o/raw_data_models_final/StackedEnsemble_AllModels_1_AutoML_1_20250125_200219'
test = pd.read_csv('/kaggle/input/test-pss5e1/test_pss5e1.csv')
test['time_id'] = test['country'] + '_' + test['product'] + '_' + test['store']
features = [col for col in test.columns if (col!='num_sold' and col!='date' and col!='id')]
cols_to_drop = ['date', 'id']
test.drop(columns=cols_to_drop, inplace=True)

In [ ]:
import h2o
h2o.init()
leader_model = h2o.load_model(best_model)

In [ ]:
h2o_df = h2o.H2OFrame(test)
preds = leader_model.predict(h2o_df)

In [ ]:
preds = preds['predict']

In [ ]:
from scipy.stats import boxcox
train = pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
train.dropna(subset='num_sold', inplace=True)
vals, lmbda = boxcox(train['num_sold'])

In [ ]:
preds = preds.as_data_frame()
preds = preds['predict'].values

In [ ]:
def inverse_boxcox(y_transformed, lmbda):
    if lmbda==0:
        return np.exp(y_transformed)
    else:
        return np.power(y_transformed*lmbda+1, 1/lmbda)
# preds = inverse_quantile(preds)
preds = inverse_boxcox(preds, lmbda)

In [ ]:
sub_test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
idx = sub_test['id']

In [ ]:
submission_df = pd.DataFrame({
    'id':idx,
    'num_sold':preds
})
submission_df.to_csv('submission.csv', index=False)